In [1]:
import torch
import json
import random

import numpy as np
import torch.nn.functional as F

from tqdm import tqdm
from random import randrange
from transformers import BertForNextSentencePrediction, AutoTokenizer

In [2]:
RANDOM_SEED = 42
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

MAX_SEQUENCE_LENGTH = 256

MODEL_PATH = 'data/best_model_state_10k_8ep_title_desc.bin'
BOOKS_PATH = 'data/top_200000_eng_books_filtered.json'
NUM_OF_BOOKS_TO_USE = 10000

In [3]:
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Model and tokenizer loading

In [4]:
model = BertForNextSentencePrediction.from_pretrained(PRE_TRAINED_MODEL_NAME)
model.load_state_dict(torch.load(MODEL_PATH))
model = model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Books loading

In [5]:
def extract_first_n_books(n):
    with open(BOOKS_PATH, 'r') as books_file:
        books = json.load(books_file)

    all_book_ids = list(books.keys())
    books_to_use = {}
    for idx in range(n):
        book_id = all_book_ids[idx]
        books_to_use[book_id] = books[book_id]

    return books_to_use, all_book_ids[:n]

In [6]:
books, book_ids = extract_first_n_books(NUM_OF_BOOKS_TO_USE)
print(f'{len(books)} books loaded.')

10000 books loaded.


# Books similar to a source one

In [7]:
source_book = book_ids[randrange(len(books))]
source_book = books[source_book]

print(f'Source book selected: {source_book}')

Source book selected: {'book_id': '12429335', 'title': 'Water Sings Blue: Ocean Poems', 'description': 'Come down to the shore with this rich and vivid celebration of the ocean! With watercolors gorgeous enough to wade in by award-winning artist Meilo So and playful, moving poems by Kate Coombs, Water Sings Blue evokes the beauty and power, the depth and mystery, and the endless resonance of the sea.', 'similar_books': ['18166919', '12962487']}


In [12]:
def get_top_k_most_similar(source_book, book_ids, books_data, k=10):
    results = []
    for book_id in tqdm(book_ids):
        book = books_data[book_id]

        encoded_sequences = tokenizer(
            source_book['description'],
            book['description'],
            padding='longest',
            truncation='longest_first',
            return_tensors='pt',
            max_length=MAX_SEQUENCE_LENGTH
        )

        outputs = model(
            input_ids=encoded_sequences['input_ids'].to(device),
            attention_mask=encoded_sequences['attention_mask'].to(device),
            token_type_ids=encoded_sequences['token_type_ids'].to(device)
        )
        probs = F.softmax(outputs.logits, dim=1)

        # Class 0 notes if sent B follows sent A
        similarity_score = probs[0][0]
        results.append((book_id, similarity_score.detach().item()))
    
    results.sort(key=lambda pair: pair[1], reverse=True)
    return results[:k]

In [13]:
similar_books = get_top_k_most_similar(source_book, book_ids, books, 10)

100%|██████████| 10000/10000 [05:47<00:00, 28.79it/s]


In [14]:
print(f'Source book title: {source_book["title"]}')
print(f'Source book description: {source_book["description"]}\n\n')

print('Top 10 most similar books:\n')
for idx, (book_id, score) in enumerate(similar_books):
    similar_book = books[book_id]
    print(f'{idx + 1}.')
    print(f'Title: {similar_book["title"]}')
    print(f'Score 0-1: {score}')
    print(f'Description: {similar_book["description"]}\n')

Source book title: Water Sings Blue: Ocean Poems
Source book description: Come down to the shore with this rich and vivid celebration of the ocean! With watercolors gorgeous enough to wade in by award-winning artist Meilo So and playful, moving poems by Kate Coombs, Water Sings Blue evokes the beauty and power, the depth and mystery, and the endless resonance of the sea.


Top 10 most similar books:

1.
Title: Water Sings Blue: Ocean Poems
Score 0-1: 0.9998713731765747
Description: Come down to the shore with this rich and vivid celebration of the ocean! With watercolors gorgeous enough to wade in by award-winning artist Meilo So and playful, moving poems by Kate Coombs, Water Sings Blue evokes the beauty and power, the depth and mystery, and the endless resonance of the sea.

2.
Title: Whitaker on Watercolor
Score 0-1: 0.9998642206192017
Description: An instruction book of watercolor techniques by watercolorist Frederic Whitaker.

3.
Title: Creative Coloring Inspirations: Art Activity

# Books similar to a description

In [15]:
description = "Inventor Tony Stark is no ordinary billionaire. He may have a big house and lots of money, but he also has a powerful suit that he wears to help fight evil and protect people in need. Tony Stark is the Invincible Iron Man!"

In [16]:
def get_top_k_most_similar_for_description(description, book_ids, books_data, k=10):
    results = []
    for book_id in tqdm(book_ids):
        book = books_data[book_id]

        encoded_sequences = tokenizer(
            description,
            book['description'],
            padding='longest',
            truncation='longest_first',
            return_tensors='pt',
            max_length=MAX_SEQUENCE_LENGTH
        )

        outputs = model(
            input_ids=encoded_sequences['input_ids'].to(device),
            attention_mask=encoded_sequences['attention_mask'].to(device),
            token_type_ids=encoded_sequences['token_type_ids'].to(device)
        )
        probs = F.softmax(outputs.logits, dim=1)

        # Class 0 notes if sent B follows sent A
        similarity_score = probs[0][0]
        results.append((book_id, similarity_score.detach().item()))
    
    results.sort(key=lambda pair: pair[1], reverse=True)
    return results[:k]

In [17]:
similar_books_to_desc = get_top_k_most_similar_for_description(description, book_ids, books, 10)

100%|██████████| 10000/10000 [05:36<00:00, 29.75it/s]


In [18]:
print(f'Source description: {description}')

print('Top 10 most similar books based on the description:\n')
for idx, (book_id, score) in enumerate(similar_books_to_desc):
    similar_book = books[book_id]
    print(f'{idx + 1}.')
    print(f'Title: {similar_book["title"]}')
    print(f'Score 0-1: {score}')
    print(f'Description: {similar_book["description"]}\n')

Source description: Inventor Tony Stark is no ordinary billionaire. He may have a big house and lots of money, but he also has a powerful suit that he wears to help fight evil and protect people in need. Tony Stark is the Invincible Iron Man!
Top 10 most similar books based on the description:

1.
Title: The Invincible Iron Man, Vol. 11: The Future
Score 0-1: 0.9998751878738403
Description: Tony Stark has always been obsessed with the future. Whether starting his first company, designing the Mark I Iron Man armor or launching Stark Resilient, his optimism has always been guided by the question, "What comes next?" Six months after quitting Resilient, he's found the answer. The sanctity of his mind violated by the Mandarin, he is a man beaten down to his knees, a prisoner within his own body, an indentured servant to an insane cause known only to his captor. For Tony Stark, this is the future - and there's no way out. COLLECTING: Invincible Iron Man 521-527

2.
Title: The Protectors
Scor